### Import Modules

In [24]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.features import ColorLine
import rasterstats
import json
import geopy
import branca.colormap as cm
import math

# Initialize map

In [109]:
# Determine Map center
UW = [47.655548, -122.303200]

#Create the map
m = folium.Map(location = UW, zoom_start = 12)
m

# Read shp file, extract route45 shp for next step to split into linestrings

In [71]:
fp = '../data/Transit_Routes_for_King_County_Metro__transitroute_line.shp'
data = gpd.read_file(fp)
dat = data[data['IN_SERVICE']=='Y']
da = dat[data['ROUTE_NUM'] == 45]
da

,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry
140,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,(LINESTRING (-122.3075605352953 47.65053260749...


# break down multi linestring to single linestring, save it as lines.shp

In [72]:
import fiona
from shapely.geometry import  MultiLineString, mapping, shape
with fiona.open('da.shp') as source:
    # create the new shapefile
    with fiona.open('lines.shp','w', driver='ESRI Shapefile',crs=source.crs,schema=source.schema) as ouput:
        # iterate the features of the original shapefile
        for elem in source:
               # iterate the list of geometries in one element (split the MultiLineString)
            for line in shape(elem['geometry']): 
                # write the line to the new shapefile
                ouput.write({'geometry':mapping(line),'properties':elem['properties']})

In [76]:
gpd.read_file('lines.shp')

,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry
0,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...
1,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...
2,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3107275034277 47.652094806596...
3,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...
4,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...
5,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3124628890345 47.652984815362...
6,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...
7,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...
8,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132461109814 47.656834269145...
9,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132070900588 47.658314904223...


# Try to get elevation and get maximum elevation with every linestring

In [54]:
sh= rasterstats.point_query('lines.shp', '../data/sea_north.tif')
SH = pd.DataFrame(sh)
SH

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,71.715242,70.969807,72.180500,72.928444,72.984795,72.104232,68.846062,64.736533,67.002208,84.540002,...,72.270787,81.465674,74.216336,72.681959,72.6613,75.567952,72.31436,72.237926,71.494447,71.715242
1,71.715242,70.238974,69.976963,67.718316,60.484874,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60.484874,91.372664,96.856224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,96.856224,91.029664,60.484874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,96.856224,71.509512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,71.509512,70.440919,69.619129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,69.619129,71.509512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,69.619129,68.599417,69.142960,74.387228,106.683444,114.145976,116.879637,129.678151,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,129.678151,138.150529,155.162508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,155.162508,130.139296,129.678151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
SH.T.max()

0       92.129986
1       71.715242
2       96.856224
3       96.856224
4       96.856224
5       71.509512
6       71.509512
7      129.678151
8      155.162508
9      155.162508
10     183.864970
11     183.864970
12     195.938721
13     195.938721
14     212.699334
15     213.872869
16     223.137782
17     256.234567
18     247.567003
19     248.266544
20     231.175765
21     256.234567
22     256.576504
23     231.175765
24     232.593040
25     256.234567
26     252.505608
27     234.731567
28     232.593040
29     232.569888
          ...    
105    339.405495
106    339.382499
107    339.077881
108    339.805326
109    339.465163
110    339.465163
111    339.465163
112    339.044061
113    339.044061
114    335.556096
115    335.556096
116    335.556096
117    338.153531
118    321.197336
119    321.197336
120    303.316220
121    312.687266
122    312.687266
123    312.687266
124    308.334072
125    308.334072
126    307.694467
127    308.259650
128    314.421066
129    289

# get points, calculate distance between two points in line segment

In [46]:
sh= rasterstats.zonal_stats('lines.shp', '../data/sea_north.tif')
SH = pd.DataFrame(sh)
SH

,count,max,mean,min
0,811,99.969002,70.771164,60.937389
1,310,119.307007,67.954095,58.150391
2,78,102.046722,82.531062,60.512062
3,78,100.254448,82.134315,60.512062
4,93,100.451881,75.590343,68.369812
5,75,89.578796,72.041016,69.613693
6,75,89.578796,72.034206,69.613693
7,342,140.569778,98.127821,68.442673
8,146,184.838211,147.069657,128.673935
9,146,184.838211,147.066553,128.673935


In [295]:
from shapely.geometry import mapping
shapefile = gpd.read_file("da.shp")
# extract the geometry in GeoJSON format
geoms = shapefile.geometry.values # list of shapely geometries
geoms = [mapping(geoms[0])]
geoms[0]['coordinates'][134]

((-122.39685586917749, 47.69056884234347),
 (-122.39653140120187, 47.690569454795934),
 (-122.39592672274738, 47.6905703444848))

In [344]:
geoms[0]['coordinates'][1]

((-122.30756053529534, 47.65053260749295),
 (-122.30820251335712, 47.65086210845893),
 (-122.30857526433199, 47.65102755761703),
 (-122.30923583856739, 47.65133981043927),
 (-122.31072750342773, 47.652094806596885))

In [475]:
linestring = []
for i in range(len(r)):
    for j in range(len(geoms[0]['coordinates'][i])):
        linestring.append(geoms[0]['coordinates'][i][j])

In [483]:
li = []
for i in range(len(geoms[0]['coordinates'])):
    li.append(len(geoms[0]['coordinates'][i]))
    
group = np.array(li)

In [439]:
from shapely.geometry import LineString
def make_lines(gdf, df_out, i, geometry = 'geometry'):    
    geom0 = linestringDF.loc[i][2]
    geom1 = linestringDF.loc[i+1][2]
    start, end = [geom0, geom1]
    line = LineString([start, end])
    
    # Create a DataFrame to hold record
    data = {'id': i,'grade':i,
            'geometry':[line]}
    df_line = pd.DataFrame(data, columns = ['id','grade', 'geometry'])
    
    # Add record DataFrame of compiled records
    df_out = pd.concat([df_out, df_line])
    return df_out

In [440]:
linestringDF = pd.DataFrame({'coor':linestring},columns=['id','grade','coor'])
for i in range(len(linestringDF)):
    linestringDF.loc[i][0] = i
linestringDF.loc[0][2]

(-122.30756053529534, 47.65053260749295)

In [566]:
geoms[0]['coordinates'][0][2]

(-122.30722583308294, 47.65059260283979)

In [ ]:
end = []
df = pd.DataFrame(columns = ['id','grade', 'geometry'])
x = 0 
while x<len(linestringDF) -1:
    for j in range(len(r)):
        for i in range(j):
            t = len(geoms[0]['coordinates'][i])
            end.append(t)
            sum = np.sum(end)
    if x == sum:
        x = x+1
    else:
        df = make_lines(linestringDF,df,x)
        x = x+1
df

In [561]:
end = []
j =2
x=0
for j in range(len(r)):
    for i in range(j):
        t = len(geoms[0]['coordinates'][i])
        end.append(t)
        sum = np.sum(end)
if x == sum:
    x = x+1
else:
    df = make_lines(linestringDF,df,x)
    x = x+1
df

,id,grade,geometry
0,0,0,LINESTRING (-122.3075605352953 47.650532607492...
0,1,1,LINESTRING (-122.3074727318998 47.650570213082...
0,2,2,LINESTRING (-122.3072258330829 47.650592602839...
0,3,3,LINESTRING (-122.3068467314136 47.650560765602...
0,4,4,LINESTRING (-122.3068312928589 47.650560143135...
0,5,5,LINESTRING (-122.3065875648685 47.650551232044...
0,6,6,LINESTRING (-122.3063251050699 47.650554352899...
0,7,7,LINESTRING (-122.3061037630416 47.650562150965...
0,8,8,LINESTRING (-122.3058318374172 47.650589248717...
0,9,9,LINESTRING (-122.3056157336896 47.650624398824...


In [559]:
end = []
j = 2
for i in range(j):
    t = len(geoms[0]['coordinates'][i])
    end.append(t)
    sum = np.sum(end)
end

[36, 5]

In [443]:
linestringGDF = gpd.GeoDataFrame(df)
linestringGDF

,id,grade,geometry
0,0,0,LINESTRING (-122.3075605352953 47.650532607492...
0,1,1,LINESTRING (-122.3074727318998 47.650570213082...
0,2,2,LINESTRING (-122.3072258330829 47.650592602839...
0,3,3,LINESTRING (-122.3068467314136 47.650560765602...
0,4,4,LINESTRING (-122.3068312928589 47.650560143135...
0,5,5,LINESTRING (-122.3065875648685 47.650551232044...
0,6,6,LINESTRING (-122.3063251050699 47.650554352899...
0,7,7,LINESTRING (-122.3061037630416 47.650562150965...
0,8,8,LINESTRING (-122.3058318374172 47.650589248717...
0,9,9,LINESTRING (-122.3056157336896 47.650624398824...


In [42]:
from geopy.distance import geodesic
geodesic([47.65086210845893,-122.30820251335712],[47.65102755761703,-122.30857526433199]).m

33.50517341257214

#  plot line segments

In [82]:
r = gpd.read_file('lines.shp')
R = r.to_json()
r45 = folium.GeoJson(R)
m.add_child(r45)
m

# below are pointless for now

# Geojson Map add max elevation (as elevation_max) for each line shape to GDF and a covert to GeoJson, then include in the style function

In [83]:
r['elevation_max'] = SH.T.max()

In [91]:
ele_min = SH.T.min()
ele_min

0       61.121241
1       60.484874
2       60.484874
3       60.484874
4       71.509512
5       69.619129
6       69.619129
7       68.599417
8      129.678151
9      129.678151
10     155.162508
11     155.162508
12     183.864970
13     177.683419
14     195.938721
15     195.938721
16     212.699334
17     248.266544
18     212.699334
19     223.137782
20     223.137782
21     248.266544
22     252.505608
23     223.137782
24     231.175765
25     252.505608
26     247.145972
27     231.175765
28     232.569888
29     225.174791
          ...    
105    338.823497
106    338.823497
107    338.823497
108    339.077881
109    339.077881
110    339.044061
111    338.966526
112    327.450368
113    327.450368
114    317.802427
115    334.016220
116    334.016220
117    321.197336
118    299.157711
119    299.157711
120    296.928320
121    301.968826
122    301.968826
123    308.334072
124    307.694467
125    307.694467
126    306.127784
127    306.127784
128    289.135245
129    276

In [98]:
r

,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry,elevation_max
0,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...,92.129986
1,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...,71.715242
2,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3107275034277 47.652094806596...,96.856224
3,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...,96.856224
4,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...,96.856224
5,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3124628890345 47.652984815362...,71.509512
6,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...,71.509512
7,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...,129.678151
8,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132461109814 47.656834269145...,155.162508
9,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132070900588 47.658314904223...,155.162508


## Try to output route45.json

In [296]:
R = r.to_json()
R

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"OBJECTID": 141, "CHANGE_NUM": 140, "MINOR_CHAN": 9, "CURRENT_NE": "IN SERVICE", "IN_SERVICE": "Y", "ROUTE_ID": 100225, "LOCAL_EXPR": "L", "ROUTE_NUM": 45, "SHAPE_Leng": 63439.57741780063, "elevation_max": 92.12998605289548}, "geometry": {"type": "LineString", "coordinates": [[-122.30756053529534, 47.65053260749295], [-122.30747273189975, 47.6505702130828], [-122.30722583308294, 47.65059260283979], [-122.3068467314136, 47.65056076560248], [-122.30683129285894, 47.650560143135365], [-122.30658756486848, 47.65055123204429], [-122.30632510506987, 47.65055435289991], [-122.30610376304159, 47.65056215096528], [-122.30583183741717, 47.65058924871735], [-122.3056157336896, 47.65062439882471], [-122.3054930531022, 47.65064846859098], [-122.30535077679728, 47.65068266374587], [-122.30510921961915, 47.65075049693666], [-122.30485900737327, 47.65084229615551], [-122.30463960044038, 47.65093287547253], [-122.

In [297]:
with open('Route_45.json', 'w') as json_file:
    json.dump(R, json_file)

In [87]:
import branca.colormap as cm
import math

Seattle_coords = [47.655548, -122.303200]
# f2 = folium.Figure(height = 400)
m2 = folium.Map(location = Seattle_coords, zoom_start = 12)

max_elev = max(r['elevation_max'])
linear = cm.linear.Paired_12.scale(0, max_elev )
gps_lyr = folium.GeoJson(R, style_function = lambda feature: {
                             'color': linear(feature['properties']['elevation_max']),
                             'weight': 5})

gps_lyr.add_child

m2.add_child(linear)
m2.add_child(gps_lyr)
# m2.add_to(f2)

In [93]:
import branca.colormap as cm
import math

Seattle_coords = [47.655548, -122.303200]
# f2 = folium.Figure(height = 400)
m2 = folium.Map(location = Seattle_coords, zoom_start = 12)

max_elev = max(r['elevation_max'])
linear = cm.linear.RdBu_06.scale(ele_min.min(), max_elev )
gps_lyr = folium.GeoJson(R, style_function = lambda feature: {
                             'color': linear(feature['properties']['elevation_max']),
                             'weight': 5})

gps_lyr.add_child

m2.add_child(linear)
m2.add_child(gps_lyr)
# m2.add_to(f2)

In [ ]:
fp = '../data/Transit_Routes_for_King_County_Metro__transitroute_line.shp'
data = gpd.read_file(fp)
dat = data[data['IN_SERVICE']=='Y']
da = dat[data['ROUTE_NUM'] == 45]
da

In [445]:
r

,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry,elevation_max
0,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...,92.129986
1,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3075605352953 47.650532607492...,71.715242
2,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3107275034277 47.652094806596...,96.856224
3,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...,96.856224
4,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3115149015316 47.652484919657...,96.856224
5,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3124628890345 47.652984815362...,71.509512
6,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...,71.509512
7,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132232503292 47.653393638881...,129.678151
8,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132461109814 47.656834269145...,155.162508
9,141,140,9,IN SERVICE,Y,100225,L,45,63439.577418,LINESTRING (-122.3132070900588 47.658314904223...,155.162508


In [251]:
b = r[r['elevation_max']==r['elevation_max'][0]]
B = b.to_json()
B.feature['properties']['elevation_max']

AttributeError: 'str' object has no attribute 'feature'

In [237]:

Seattle_coords = [47.655548, -122.303200]
# f2 = folium.Figure(height = 400)
m2 = folium.Map(location = Seattle_coords, zoom_start = 12)

max_elev = max(r['elevation_max'])
linear = cm.linear.RdBu_06.scale(ele_min.min(), max_elev )
gps_lyr = folium.GeoJson(B, style_function = lambda feature: {
                             'color': linear(feature['properties']['elevation_max']),
                             'weight': 5})

gps_lyr.add_child

m2.add_child(linear)
m2.add_child(gps_lyr)
# m2.add_to(f2)

# plot using points and linestring. If points<x, use points to draw.

In [494]:
Seattle_coords = [47.655548, -122.303200]
# f2 = folium.Figure(height = 400)
m2 = folium.Map(location = Seattle_coords, zoom_start = 12)
max_elev = max(r['elevation_max'])
linear = cm.linear.RdBu_06.scale(ele_min.min(), max_elev )
points = linestringGDF.to_json()
for i in range(len(r)):
    b = r[r['elevation_max']==r['elevation_max'][i]]
    onelinestring = b.to_json()
    if len(geoms[0]['coordinates'][i])<2:

        gps_lyr = folium.GeoJson(points, style_function = lambda feature: {
            'color': linear(feature['properties']['grade']), 'weight': 5})
        gps_lyr.add_child
        m2.add_child(linear)
        m2.add_child(gps_lyr)
        
    else:
        gps_lyr = folium.GeoJson(onelinestring, style_function = lambda feature: {
            'color': linear(feature['properties']['elevation_max']), 'weight': 5})
        gps_lyr.add_child
        m2.add_child(linear)
        m2.add_child(gps_lyr)
        # m2.add_to(f2)
m2

In [525]:
last = len(geoms[0]['coordinates'][i])-1

geoms[0]['coordinates'][i][last]

(-122.30683129285894, 47.650560143135365)

In [531]:
linestringGDF

,id,grade,geometry
0,0,0,LINESTRING (-122.3075605352953 47.650532607492...
0,1,1,LINESTRING (-122.3074727318998 47.650570213082...
0,2,2,LINESTRING (-122.3072258330829 47.650592602839...
0,3,3,LINESTRING (-122.3068467314136 47.650560765602...
0,4,4,LINESTRING (-122.3068312928589 47.650560143135...
0,5,5,LINESTRING (-122.3065875648685 47.650551232044...
0,6,6,LINESTRING (-122.3063251050699 47.650554352899...
0,7,7,LINESTRING (-122.3061037630416 47.650562150965...
0,8,8,LINESTRING (-122.3058318374172 47.650589248717...
0,9,9,LINESTRING (-122.3056157336896 47.650624398824...
